# CAPSTONE Project - Neighborhood Clustering

### Importing the dataset - IBM Watson

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_a37a82f472884de3af85202afee14c1c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='umibtusPlqLR-EnYKJqqlcguAERerzijYa3ae_mtuLKO',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a37a82f472884de3af85202afee14c1c.get_object(Bucket='capstonetoronto-donotdelete-pr-6kve0jrbutzhnh',Key='Capstone_project.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### The folloiwng is the shape of the dataset

In [3]:
df_data_1.shape

(288, 3)

### Examining how may boroughs have "Not Assigned" status

In [4]:
df_data_1.Borough.value_counts()

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

#### In the Dataset, 77 entries of borroughs have "Not Assigned Status"

### Since the boroughs which have "Not assgined" status needs to be ignored, I need to drop the same from the dataset. For this, the indexes of such "Not assigned" status is found out. There should be total 77 such entries.

In [5]:
indexnames = df_data_1[df_data_1['Borough'] == "Not assigned"].index
indexnames

Int64Index([  0,   1,   9,  13,  20,  21,  30,  36,  37,  45,  46,  50,  51,
             52,  54,  55,  59,  60,  61,  73,  74,  75,  88,  89,  90, 104,
            105, 106, 120, 121, 136, 137, 148, 149, 155, 161, 162, 167, 175,
            181, 182, 188, 189, 190, 194, 195, 201, 202, 203, 204, 209, 210,
            223, 224, 237, 238, 241, 242, 247, 248, 253, 254, 258, 259, 260,
            261, 263, 264, 274, 275, 276, 277, 278, 279, 280, 281, 287],
           dtype='int64')

### We drop these entries where Boroughs have "Not assigned" status rom the dataset. Since original dataset had 288 rows and 77 of them were deleted, there would be 211 rows. We check the same

In [6]:
df_data_1.drop(indexnames, axis=0, inplace = True)
df_data_1.shape

(211, 3)

### We once again check if our exercise have been succesful.

In [7]:
df_data_1.Borough.value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

#### There are no "Not Assigned" Boroughs in the resulting dataset.

### Next, the number of unique Postcodes was found.

In [8]:
print("Number of Unique Postal codes is :", len(df_data_1.Postcode.unique()))

Number of Unique Postal codes is : 103


### Next, we check if there are any neighbourhood/s which have "Not Assigned" Status

In [9]:
df_data_1[df_data_1.Neighbourhood =='Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


#### There is one entry where Neighbourhood has "Not assigned" status. We need to replace the Neighbourhood value with Borough name, which we do using the below code.

In [10]:
df_data_1.Neighbourhood.replace("Not assigned", "Queen's Park", inplace = True)

#### We check whether the same has been replaced.

In [11]:
df_data_1[df_data_1['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


### The neighbourhoods are joined using the below mentioned code using group by and lambda. It is saved in a different dataframe

In [12]:
df= pd.DataFrame(df_data_1.groupby("Postcode")['Neighbourhood'].apply(lambda tags: ','.join(tags)))
df

,Neighbourhood
Postcode,
M1B,"Rouge,Malvern"
M1C,"Highland Creek,Rouge Hill,Port Union"
M1E,"Guildwood,Morningside,West Hill"
M1G,Woburn
M1H,Cedarbrae
M1J,Scarborough Village
M1K,"East Birchmount Park,Ionview,Kennedy Park"
M1L,"Clairlea,Golden Mile,Oakridge"
M1M,"Cliffcrest,Cliffside,Scarborough Village West"


### Groupby statment results in indexing the Key. The same can be seen as below.

In [13]:
df.head(5)

,Neighbourhood
Postcode,
M1B,"Rouge,Malvern"
M1C,"Highland Creek,Rouge Hill,Port Union"
M1E,"Guildwood,Morningside,West Hill"
M1G,Woburn
M1H,Cedarbrae


### The indexing is removed below.

In [14]:
df.reset_index(inplace = True)
df.head(5)

,Postcode,Neighbourhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


### Next, we need to merge the 2 dataframes on Postcode for merging Neighbourhood. Hnece first, we drop the "Neighbourhood column from the original dataframe.

In [15]:
df_data_1.drop('Neighbourhood', axis= 1, inplace =True)
df_data_1.head(5)

,Postcode,Borough
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
5,M5A,Downtown Toronto
6,M6A,North York


### Merging the two dataframes.

In [16]:
df_final = pd.merge(df_data_1, df, on= 'Postcode')
df_final

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge,Malvern"
9,M1B,Scarborough,"Rouge,Malvern"


### Sicne there were 211 rows in the original dataframe, the resulting merged dataframe will also contain 211 rows, as shown below. 

In [17]:
df_final.shape

(211, 3)

### However, this dataframe contains many duplicate values since the number of unqiue postcodes is only 103. We drop the duplicates ans check the final shape, which should contain 103 rows.

In [18]:
df_final.drop_duplicates(inplace = True)
df_final.shape

(103, 3)

### resetting the index

In [19]:
df_final.reset_index(inplace =True)
df_final

,index,Postcode,Borough,Neighbourhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,4,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,6,M7A,Queen's Park,Queen's Park
5,7,M9A,Etobicoke,Islington Avenue
6,8,M1B,Scarborough,"Rouge,Malvern"
7,10,M3B,North York,Don Mills North
8,11,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,13,M5B,Downtown Toronto,"Ryerson,Garden District"


### Removing the index column,

In [20]:
df_final.drop('index', axis=1, inplace = True)
df_final

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


## The final shape of the dataframe:

In [109]:
df_final.shape

(103, 3)

# The final dataframe consists of 103 rows and 3 columns

###  

# Clustering and exploring neighbourhood

### Loading the dataset. The same is done using the Geospatial_Cordinates.csv provided by the link in the "My Submission" Page

In [21]:

body = client_a37a82f472884de3af85202afee14c1c.get_object(Bucket='capstonetoronto-donotdelete-pr-6kve0jrbutzhnh',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Finding the shape of the dataset. 

In [22]:
df_data_3.shape

(103, 3)

### Sicne the two datasets have to be merged on Postcode- the key, we rename the column in the new dataframe by Postcode.

In [23]:
df_data_3 = df_data_3.rename(columns = {"Postal Code": "Postcode"})
df_data_3.head(5)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two dataframes

In [24]:
df_latlon = pd.merge(df_final,df_data_3,on='Postcode')
df_latlon.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### Using geopy library to get the latitude and longitude values of Toronto.

In [25]:
from geopy.geocoders import Nominatim

In [26]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [27]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

### Creating a map of Toronto with neighborhoods superimposed on top.

In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df_latlon['Latitude'], df_latlon['Longitude'], df_latlon['Borough'], df_latlon['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Checking the number of Boroughs in the dataset.

In [29]:
df_latlon.Borough.value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### Since North York has the highest number of entries, it is decided to explore North York. A separate dataframe is created for North York only

In [30]:
northyork_data =df_latlon[df_latlon['Borough']=='North York'].reset_index(drop = True)
northyork_data.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


### Removing the Postcode from the dataframe, no longer required.

In [31]:
northyork_data.drop('Postcode', axis= 1, inplace= True)
northyork_data.head(5)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,North York,Don Mills North,43.745906,-79.352188
4,North York,Glencairn,43.709577,-79.445073


### Getting the geograhical coordinates of North York

In [32]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.7708175, -79.4132998.


### Creating a map of North York with neighbourhoods in it.

In [33]:
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

### Defining Foursquare Credentials and Version

In [34]:
CLIENT_ID = 'TVMEP44S5OUTVYLHCLQLSMI0TF3YD2WBUU2S5AILD55EHUBN' 
CLIENT_SECRET = 'PQMZV25AASDDEW13LH4YSD1O4MCYHDLCC10J4OHKMG2IP0CT' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TVMEP44S5OUTVYLHCLQLSMI0TF3YD2WBUU2S5AILD55EHUBN
CLIENT_SECRET:PQMZV25AASDDEW13LH4YSD1O4MCYHDLCC10J4OHKMG2IP0CT


### Exploring the first neighbourhood in the dataset.

In [35]:
northyork_data.loc[0, 'Neighbourhood']

'Parkwoods'

### Getting the neighbourhoods cordinates

In [36]:
neighbourhood_latitude = northyork_data.loc[0, 'Latitude'] 
neighbourhood_longitude = northyork_data.loc[0, 'Longitude'] 

neighbourhood_name = northyork_data.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### top 100 venues that are in Parkwoodsl within a radius of 500 meters

In [37]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TVMEP44S5OUTVYLHCLQLSMI0TF3YD2WBUU2S5AILD55EHUBN&client_secret=PQMZV25AASDDEW13LH4YSD1O4MCYHDLCC10J4OHKMG2IP0CT&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=100'

### GET request and examining the resutls

In [38]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas datafram

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cde3e641ed21914c03db060'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'location': {'address': 'Toronto',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 245,
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'lat

### get_category_type function.

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### clean the json and structure it into a pandas dataframe.

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,GreenWin pool,Pool,43.756232,-79.333842


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in North York

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### A new dataframe is created.

In [43]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighbourhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights,Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park,Don Mills South
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
CFB Toronto,Downsview East
Silver Hills,York Mills
Downsview West
Downsview,North Park,Upwood Park
Humber Summit
Newtonbrook,Willowdale
Downsview Central
Bedford Park,Lawrence Manor East
Emery,Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


### size of the resulting dataframe

In [44]:
print(northyork_venues.shape)
northyork_venues.head()

(243, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,GreenWin pool,43.756232,-79.333842,Pool
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


### venues for each neighborhood

In [45]:
northyork_venues.groupby('Neighborhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto,Downsview East",3,3,3,3,3,3
Don Mills North,4,4,4,4,4,4
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


### unique categories in the dataset

In [46]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 108 uniques categories.


### Analyze Each Neighborhood

In [47]:
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")


northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### the new dataframe size

In [48]:
northyork_onehot.shape

(243, 109)

### grouping rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [49]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,...,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto,Downsview East",0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview,North Park,Upwood Park",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery,Humberlea",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### The size of the new dataframe

In [50]:
northyork_grouped.shape

(23, 109)

### Each neighborhood along with the top 5 most common venues

In [51]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
              venue  freq
0       Coffee Shop  0.11
1  Sushi Restaurant  0.05
2        Restaurant  0.05
3    Sandwich Place  0.05
4     Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2  Japanese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.09
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.09
3         Grocery Store  0.04
4               Butcher  0.04


----CFB Toronto,Downsview East----
         venue  freq
0      Airport  0.33
1         Park  0.33
2  Snack Place  0.33
3     Pharmacy  0.00
4    Pet Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4        Massage Studio  0.0

### Putting the same into a pandas dataframe

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating the new dataframe and displaying the top 10 venues for each neighborhood.

In [53]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fried Chicken Joint,Shopping Mall,Middle Eastern Restaurant,Frozen Yogurt Shop,Pet Store,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Indian Restaurant,Café,Liquor Store,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Snack Place,Airport,Park,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


### Run k-means to cluster the neighborhood into 5 clusters.

In [54]:
from sklearn.cluster import KMeans

kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 1, 4, 0, 0, 0, 0, 2], dtype=int32)

### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [56]:
northyork_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

northyork_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Pool,Food & Drink Shop,Fast Food Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
1,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
2,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Gift Shop,Accessories Store
3,North York,Don Mills North,43.745906,-79.352188,4.0,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Women's Store,Electronics Store,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
4,North York,Glencairn,43.709577,-79.445073,0.0,Arcade,Bakery,Pub,Japanese Restaurant,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


### Checking whether there are any NaN entries

In [83]:
northyork_merged.isnull().sum()

Borough                   0
Neighbourhood             0
Latitude                  0
Longitude                 0
Cluster Labels            1
1st Most Common Venue     1
2nd Most Common Venue     1
3rd Most Common Venue     1
4th Most Common Venue     1
5th Most Common Venue     1
6th Most Common Venue     1
7th Most Common Venue     1
8th Most Common Venue     1
9th Most Common Venue     1
10th Most Common Venue    1
dtype: int64

### Seems like there is one entry with Nan value. Checking which entry it is.

In [96]:
northyork_merged[northyork_merged['10th Most Common Venue'].isnull()]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dropping the NaN netity from the dataset.

In [101]:
northyork_merged.dropna(axis =0, inplace = True)
northyork_merged.isnull().sum()

Borough                   0
Neighbourhood             0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

### Visualizing the resulting clusters

In [103]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighbourhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### Examining Clusters

#### Cluster 1

In [104]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Pool,Food & Drink Shop,Fast Food Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
1,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
2,"Lawrence Heights,Lawrence Manor",Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Gift Shop,Accessories Store
4,Glencairn,Arcade,Bakery,Pub,Japanese Restaurant,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
5,"Flemingdon Park,Don Mills South",Coffee Shop,Asian Restaurant,Gym,Beer Store,Bike Shop,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Restaurant
6,Hillcrest Village,Golf Course,Pool,Athletics & Sports,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping
7,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fried Chicken Joint,Shopping Mall,Middle Eastern Restaurant,Frozen Yogurt Shop,Pet Store,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop
8,"Fairview,Henry Farm,Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Bus Station,Asian Restaurant,Juice Bar,Restaurant,Jewelry Store,Japanese Restaurant,Bakery
9,"Northwood Park,York University",Coffee Shop,Furniture / Home Store,Bar,Falafel Restaurant,Massage Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
13,Downsview West,Moving Target,Grocery Store,Bank,Shopping Mall,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


#### Cluster 2

In [105]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"CFB Toronto,Downsview East",Snack Place,Airport,Park,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
12,"Silver Hills,York Mills",Park,Cafeteria,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


#### Cluster 3

In [106]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"Emery,Humberlea",Construction & Landscaping,Baseball Field,Women's Store,Empanada Restaurant,Comfort Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


#### Cluster 4

In [107]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Humber Summit,Pizza Place,Empanada Restaurant,Dog Run,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


#### Cluster 5

In [108]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 5, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### No fifth cluster is present.